<a href="https://colab.research.google.com/github/dwu12/Machine-Learning-Project/blob/main/Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import collections
import math
import torch
from torch import nn
from torch.nn.utils import clip_grad_norm_
import matplotlib.pyplot as plt

!pip install torchmetrics

from torchmetrics.functional import bleu_score
from torchtext.data.metrics import bleu_score as bs
import numpy as np

### Dataset and DataLoader

In [2]:
path = 'https://github.com/desmondyeoh/data-eng-fra/blob/master/eng-fra.txt'

def read_data_nmt():
    """载入“英语－法语”数据集"""
    with open('/content/drive/MyDrive/DL/RNN/eng-fra.txt', 'r',
             encoding='utf-8') as f:
        return f.read()

raw_text = read_data_nmt()
print(raw_text[:75])

Go.	Va !
Run!	Cours !
Run!	Courez !
Wow!	Ça alors !
Fire!	Au feu !
Help!	À 


In [3]:
def preprocess_nmt(text):
    """预处理“英语－法语”数据集"""
    def no_space(char, prev_char):
        return char in set(',.!?') and prev_char != ' '

    # 使用空格替换不间断空格
    # 使用小写字母替换大写字母
    text = text.replace('\u202f', ' ').replace('\xa0', ' ').lower()
    # 在单词和标点符号之间插入空格
    out = [' ' + char if i > 0 and no_space(char, text[i - 1]) else char for i, char in enumerate(text)]
    return ''.join(out)

text = preprocess_nmt(raw_text)
print(text[:80])

go .	va !
run !	cours !
run !	courez !
wow !	ça alors !
fire !	au feu !
help !	à


In [4]:
def tokenize_nmt(text, num_examples=None):
    """词元化“英语－法语”数据数据集"""
    source, target = [], []
    for i, line in enumerate(text.split('\n')):
        if num_examples and i > num_examples:
            break
        parts = line.split('\t')
        if len(parts) == 2:
            source.append(parts[0].split(' '))
            target.append(parts[1].split(' '))
    return source, target

source, target = tokenize_nmt(text)
source[:6], target[:6]

([['go', '.'],
  ['run', '!'],
  ['run', '!'],
  ['wow', '!'],
  ['fire', '!'],
  ['help', '!']],
 [['va', '!'],
  ['cours', '!'],
  ['courez', '!'],
  ['ça', 'alors', '!'],
  ['au', 'feu', '!'],
  ['à', "l'aide", '!']])

In [5]:
class Vocab:
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []

        counter = self.count_corpus(tokens)
        self._token_freqs = sorted(counter.items(), key=lambda x: x[1], reverse=True)

        self.idx_to_token = ['<unk>'] + reserved_tokens
        self.token_to_idx = {token: idx
                             for idx, token in enumerate(self.idx_to_token)}
        for token, freq in self._token_freqs:
            if freq < min_freq:
                break
            if token not in self.token_to_idx:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1

    def count_corpus(self, tokens):
        if len(tokens) == 0 or isinstance(tokens[0], list):
          tokens = [token for line in tokens for token in line]
        return collections.Counter(tokens)

    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

    @property
    def unk(self):  # 未知词元的索引为0
        return 0

    @property
    def token_freqs(self):
        return self._token_freqs

In [6]:
src_vocab = Vocab(source, min_freq=2, reserved_tokens=['<pad>', '<bos>', '<eos>'])
len(src_vocab)

9339

In [7]:
def truncate_pad(line, num_steps, padding_token):
    """截断或填充文本序列"""
    if len(line) > num_steps:
        return line[:num_steps]  # 截断
    return line + [padding_token] * (num_steps - len(line))  # 填充

truncate_pad(src_vocab[source[0]], 10, src_vocab['<pad>'])

[48, 4, 1, 1, 1, 1, 1, 1, 1, 1]

In [8]:
def build_array_nmt(lines, vocab, num_steps):
    """将机器翻译的文本序列转换成小批量"""
    lines = [vocab[l] for l in lines]
    lines = [l + [vocab['<eos>']] for l in lines]
    array = torch.tensor([truncate_pad(l, num_steps, vocab['<pad>']) for l in lines])
    valid_len = (array != vocab['<pad>']).type(torch.int32).sum(1)
    return array, valid_len

In [9]:
class customer_dataset(torch.utils.data.Dataset):
    def __init__(self, features, labels):
        super(customer_dataset, self).__init__()
        self.features = features
        self.labels = labels

    def __getitem__(self, idx):
        sample, label = self.features[idx,:], self.labels[idx,:]
        return sample, label

    def __len__(self):
        return len(self.features)

In [10]:
def load_data_nmt(batch_size, num_steps, num_examples=600):
    """返回翻译数据集的迭代器和词表"""
    text = preprocess_nmt(read_data_nmt())
    source, target = tokenize_nmt(text, num_examples)

    ## Get source vocab and target vocab
    src_vocab = Vocab(source, min_freq=2, reserved_tokens=['<pad>', '<bos>', '<eos>'])
    tgt_vocab = Vocab(target, min_freq=2, reserved_tokens=['<pad>', '<bos>', '<eos>'])

    src_array, src_valid_len = build_array_nmt(source, src_vocab, num_steps)
    tgt_array, tgt_valid_len = build_array_nmt(target, tgt_vocab, num_steps)

    ### get data
    data_arrays = (src_array, src_valid_len, tgt_array, tgt_valid_len)
    dataset = customer_dataset(src_array, tgt_array)
    data_iter = torch.utils.data.DataLoader(dataset, batch_size = batch_size, shuffle=True)
    return data_iter, src_vocab, tgt_vocab

In [11]:
def show_heatmaps(matrices, xlabel, ylabel, titles=None, figsize=(2.5, 2.5),
                  cmap='Reds'):
    """显示矩阵热图"""
    num_rows, num_cols = matrices.shape[0], matrices.shape[1]
    fig, axes = plt.subplots(num_rows, num_cols, figsize=figsize,
                                 sharex=True, sharey=True, squeeze=False)
    for i, (row_axes, row_matrices) in enumerate(zip(axes, matrices)):
        for j, (ax, matrix) in enumerate(zip(row_axes, row_matrices)):
            pcm = ax.imshow(matrix.detach().numpy(), cmap=cmap)
            if i == num_rows - 1:
                ax.set_xlabel(xlabel)
            if j == 0:
                ax.set_ylabel(ylabel)
            if titles:
                ax.set_title(titles[j])
    fig.colorbar(pcm, ax=axes, shrink=0.6)
    plt.show()

### Training

In [12]:
def train_seq2seq(net, data_iter, lr, num_epochs, tgt_vocab, device):
    """训练序列到序列模型"""
    def xavier_init_weights(m):
        if type(m) == nn.Linear:
            nn.init.xavier_uniform_(m.weight)
        if type(m) == nn.GRU:
            for param in m._flat_weights_names:
                if "weight" in param:
                    nn.init.xavier_uniform_(m._parameters[param])

    net.apply(xavier_init_weights)
    net.to(device)
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)
    loss = nn.CrossEntropyLoss(ignore_index= 1)
    net.train()

    metric = []
    for epoch in range(num_epochs):
        for X, Y in data_iter:
            optimizer.zero_grad()
            X, Y = X.to(device), Y.to(device)
            bos = torch.tensor([tgt_vocab['<bos>']] * Y.shape[0], device=device).reshape(-1, 1)
            dec_input = torch.cat([bos, Y[:, :-1]], 1)  # 强制教学
            Y_hat = net(X, dec_input)
            l = loss(Y_hat.reshape(-1, Y_hat.shape[2]), Y.reshape(-1))
            l.backward()
            clip_grad_norm_(net.parameters(), 1)
            optimizer.step()
            with torch.no_grad():
                metric.append(l)

        if (epoch + 1) % 10 == 0:
            print(f'loss {sum(metric)/len(metric)}')

### Transformer

In [13]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        # Ensure that the model dimension (d_model) is divisible by the number of heads
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"

        # Initialize dimensions
        self.d_model = d_model # Model's dimension
        self.num_heads = num_heads # Number of attention heads
        self.d_k = d_model // num_heads # Dimension of each head's key, query, and value

        # Linear layers for transforming inputs
        self.W_q = nn.Linear(d_model, d_model) # Query transformation
        self.W_k = nn.Linear(d_model, d_model) # Key transformation
        self.W_v = nn.Linear(d_model, d_model) # Value transformation
        self.W_o = nn.Linear(d_model, d_model) # Output transformation

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        # Calculate attention scores
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)

        # Apply mask if provided (useful for preventing attention to certain parts like padding)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)

        # Softmax is applied to obtain attention probabilities
        attn_probs = torch.softmax(attn_scores, dim=-1)

        # Multiply by values to obtain the final output
        output = torch.matmul(attn_probs, V)
        return output

    def split_heads(self, x):
        # Reshape the input to have num_heads for multi-head attention
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)

    def combine_heads(self, x):
        # Combine the multiple heads back to original shape
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)

    def forward(self, Q, K, V, mask=None):
        # Apply linear transformations and split heads
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))

        # Perform scaled dot-product attention
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)

        # Combine heads and apply output transformation
        output = self.W_o(self.combine_heads(attn_output))
        return output

In [14]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

In [15]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()

        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

In [16]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

In [17]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

In [18]:
class Encoder(nn.Module):
  def __init__(self, d_model, num_heads, d_ff, dropout, src_vocab_size, num_layers, max_seq_length):
    super(Encoder, self).__init__()
    self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
    self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
    self.positional_encoding = PositionalEncoding(d_model, max_seq_length)
    self.dropout = nn.Dropout(dropout)
    self.d_model = d_model

  def forward(self, src, src_mask):
    src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src) * math.sqrt(self.d_model)))
    enc_output = src_embedded
    for enc_layer in self.encoder_layers:
        enc_output = enc_layer(enc_output, src_mask)

    return enc_output


class Decoder(nn.Module):
  def __init__(self, d_model, num_heads, d_ff, dropout, tgt_vocab_size, num_layers, max_seq_length):
    super(Decoder, self).__init__()
    self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
    self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
    self.positional_encoding = PositionalEncoding(d_model, max_seq_length)
    self.dropout = nn.Dropout(dropout)
    self.fc = nn.Linear(d_model, tgt_vocab_size)
    self.d_model = d_model

  def forward(self, tgt, enc_output, src_mask, tgt_mask):
    tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt) * math.sqrt(self.d_model)))

    dec_output = tgt_embedded
    for dec_layer in self.decoder_layers:
        dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

    output = self.fc(dec_output)
    return output


class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder = Encoder(d_model, num_heads, d_ff, dropout, src_vocab_size, num_layers, max_seq_length)
        self.decoder = Decoder(d_model, num_heads, d_ff, dropout, tgt_vocab_size, num_layers, max_seq_length)

    def generate_mask(self, src, tgt):
        src_mask = (src != 1).unsqueeze(1).unsqueeze(2).to('cuda')
        tgt_mask = (tgt != 1).unsqueeze(1).unsqueeze(3).to('cuda')
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool().to('cuda')
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)

        enc_output = self.encoder(src, src_mask)
        dec_output = self.decoder(tgt, enc_output, src_mask, tgt_mask)
        return dec_output

###training details

In [19]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

train_iter, src_vocab, tgt_vocab = load_data_nmt(32, 10)

In [20]:
src_vocab_size = len(src_vocab)
tgt_vocab_size = len(tgt_vocab)
d_model = 32
num_heads = 4
num_layers = 2
d_ff = 128
max_seq_length = 10
dropout = 0.1
lr, num_epochs = 0.005, 500
model = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

In [21]:
train_seq2seq(model, train_iter, lr, num_epochs, tgt_vocab, device)

loss 2.0735135078430176
loss 1.6619459390640259
loss 1.4155644178390503
loss 1.2492358684539795
loss 1.1273351907730103
loss 1.0305633544921875
loss 0.9532252550125122
loss 0.8911542892456055
loss 0.8382785320281982
loss 0.7935157418251038
loss 0.7558156847953796
loss 0.7227541208267212
loss 0.6941323280334473
loss 0.6693258285522461
loss 0.647087037563324
loss 0.6265802383422852
loss 0.6077691316604614
loss 0.5912443399429321
loss 0.576025664806366
loss 0.5620740056037903
loss 0.5493149757385254
loss 0.5376784205436707
loss 0.526922881603241
loss 0.5169119834899902
loss 0.5075514316558838
loss 0.49884966015815735
loss 0.4906163513660431
loss 0.4826175570487976
loss 0.4753783345222473
loss 0.4689069986343384
loss 0.4626474380493164
loss 0.456558495759964
loss 0.4507838189601898
loss 0.44537872076034546
loss 0.44019585847854614
loss 0.4351275563240051
loss 0.4304210841655731
loss 0.42597100138664246
loss 0.4216630160808563
loss 0.4176138639450073
loss 0.41363176703453064
loss 0.40980607

### Prediction

In [22]:
def predict_seq2seq(net, src_sentence, src_vocab, tgt_vocab, num_steps,
                    device, save_attention_weights=False):
    """序列到序列模型的预测"""
    # 在预测时将net设置为评估模式
    net.eval()
    src_tokens = src_vocab[src_sentence.lower().split(' ')] + [src_vocab['<eos>']]
    #print('what is src_tokens', src_tokens)
    enc_valid_len = torch.tensor([len(src_tokens)], device=device)
    src_tokens = truncate_pad(src_tokens, num_steps, src_vocab['<pad>'])
    #print('what is src_tokens', src_tokens)
    # 添加批量轴
    enc_X = torch.unsqueeze(torch.tensor(src_tokens, dtype=torch.long, device=device), dim=0)
    src_mask = (enc_X != 1).unsqueeze(1).unsqueeze(2).to('cuda')
    #print('what is encoder x', enc_X.shape)
    enc_outputs = net.encoder(enc_X, src_mask)
    # dec_state = net.decoder.init_state(enc_outputs, None)
    # 添加批量轴
    dec_X = torch.unsqueeze(torch.tensor([tgt_vocab['<bos>']], dtype=torch.long, device=device), dim=0)
    #print('what is decoder x', dec_X.shape)
    output_seq, attention_weight_seq = [], []
    for _ in range(num_steps):
        Y = net.decoder(dec_X, enc_outputs, src_mask, None)
        #print(Y.shape)
        #print(dec_state.shape)
        # 我们使用具有预测最高可能性的词元，作为解码器在下一时间步的输入
        dec_X = Y.argmax(dim=2)
        pred = dec_X.squeeze(dim=0).type(torch.int32).item()
        # 保存注意力权重（稍后讨论）
        # if save_attention_weights:
        #     attention_weight_seq.append(net.decoder.attention_weights)
        # 一旦序列结束词元被预测，输出序列的生成就完成了
        if pred == tgt_vocab['<eos>']:
            break
        output_seq.append(pred)
    return ' '.join(tgt_vocab.to_tokens(output_seq)), attention_weight_seq

In [24]:
engs = ['go .', "i lost .", 'he\'s calm .', 'i\'m home .']
fras = ['va !', 'j\'ai perdu .', 'il est calme .', 'je suis chez moi .']
for eng, fra in zip(engs, fras):
    translation, dec_attention_weight_seq = predict_seq2seq(
        model, eng, src_vocab, tgt_vocab, 10, device, False)
    print(f'{eng} => {translation}, bleu { bleu_score([translation], [fra], n_gram=2) :.3f}')
    A = bs( candidate_corpus = [translation.split(' ')], references_corpus= [[fra.split(' ')]], max_n=2, weights = [0.5, 0.5])
    print(f'{eng} => {translation}, bleu { A:.3f}')

go . => va !, bleu 1.000
go . => va !, bleu 1.000
i lost . => j'ai perdu ., bleu 1.000
i lost . => j'ai perdu ., bleu 1.000
he's calm . => il ., bleu 0.000
he's calm . => il ., bleu 0.000
i'm home . => je suis ., bleu 0.363
i'm home . => je suis ., bleu 0.363


/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:70: FutureWarning: Importing `bleu_score` from `torchmetrics.functional` was deprecated and will be removed in 2.0. Import `bleu_score` from `torchmetrics.text` instead.
  _future_warning(
